In [4]:
## after the finetuning folder is shared, Use the code below to mount the drive folder to colab
## Navigate to the appropriate folder to start reviewing and running the code

import os
try:
    import google.cloud.sql.connector
    import pg8000
    import bitsandbytes
except ImportError:
    !pip install -q "cloud-sql-python-connector[pg8000]"
    !pip install bitsandbytes
    print("Successfully installed and imported.")
    print("You will now need to rerun this cell")
    os.kill(os.getpid(), 9)

from google.colab import drive
drive.mount('/content/drive')
print("Mounted Drive")

# Navigate to Folder containing the finetuning files. Make sure to update as needed
%cd /content/drive/MyDrive/Colab Notebooks/NewsJuice

Mounted at /content/drive
Mounted Drive
/content/drive/MyDrive/Colab Notebooks/NewsJuice


In [2]:
%run 01_get_chucks_from_db.py

Starting Chuncks Extractions
Finished Loading Dependancies
Connecting to Cloud DB
Connecting to database...
Connection successful!
Starting Extraction
Fetching 2000 random chunks...


Fetching Chunks:   0%|          | 0/2000 [00:00<?, ?it/s]

Saving 'content' of 2000 chunks to Output/01_get_chuncks_from_db.csv...
Successfully saved 'content' to Output/01_get_chuncks_from_db.csv


In [4]:
%run 02_get_gemini_training_data.py

Starting Script
Loaded Dependancies
Found existing output file. Counting processed rows...
Found 1048 already processed data rows. Will skip.
Found input CSV and loading chunks...
Total chunks requested: 2000
Already processed: 1048
New chunks to process: 952
[gemini] Configured with Vertex AI service account authentication
[gemini] Using model: gemini-2.5-flash in us-central1

--- Running Gemini generation for 952 new chunks ---


Gemini generations: 100%|██████████| 952/952 [1:59:24<00:00,  7.53s/chunk]

[info] Script finished. Processed 952 new chunks.


In [5]:
%run 03_get_Qwen_initial_response.py

Starting Qwen Generation Without finetuning
Loaded Dependancies
Found CSV and Loading it

--- Running Qwen generation in batches ---


Qwen batches:   0%|          | 0/500 [00:00<?, ?batch/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0
Qwen batches: 100%|██████████| 500/500 [12:07:29<00:00, 87.30s/batch]


In [10]:
import torch
import gc

# 2. Call Python's garbage collector
gc.collect()

# 3. Finally, empty the PyTorch cache
torch.cuda.empty_cache()

In [11]:
%run 04_finetuning.py

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Starting Finetuning
Loaded Dependancies


eval/loss,█▂▁
eval/runtime,█▂▁
eval/samples_per_second,▁▇█
eval/steps_per_second,▁██
train/epoch,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇██████
train/global_step,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇██
train/grad_norm,▂▁▁▁▁▂▂▂▂▂▂▂▆▂▂▂▆▃▃▃▄▅▄▃▄▃█▃▂▃▃▃▃▃▃▃▄▂▃▃
train/learning_rate,▁▄██████▇▇▇▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,████▇█▇▆▆▇▆▅▄▅▄▅▄▄▄▄▃▃▂▃▂▁▃▂▃▃▂▂▃▃▂▂▃▁▂▁
eval/loss,1.35198
eval/runtime,27.6791


Starting QLoRA Fine-tuning for Qwen Model
Training Data: 2000 examples loaded from Output/02_get_gemini_training_data.csv
Model: Qwen/Qwen3-0.6B
📊 Dataset split: 1800 train / 200 validation

🔤 Tokenizing dataset...


Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

...

✅ Tokenization complete: 1800 train examples, 200 validation examples

⚙️  Compute dtype: torch.bfloat16

📥 Loading model with 4-bit quantization...
✅ Model loaded successfully

🎯 LoRA Configuration:
   - Rank (r): 64
   - Alpha: 16
   - Dropout: 0.05
   - Trainable params: 40,370,176 (9.70%)
   - Total params: 416,219,136

🏋️  Starting training...



/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,1.811100,1.813446
2,1.048000,1.262063
3,0.978500,0.896483
4,0.735800,0.698443
5,0.472100,0.596234
6,0.197600,0.546252
7,0.133500,0.526902
8,0.172500,0.520934
9,0.135500,0.527384
10,0.092300,0.530699


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/

Training completed!

Saving model and tokenizer to: qwen_qlora_podcast
Model saved successfully!

Fine-tuned adapter location: qwen_qlora_podcast
 Fine-tuning complete! You can now load the adapter with PEFT.



In [12]:
%run 05_merge_model.py

Base model: Qwen/Qwen3-0.6B
Adapter: qwen_qlora_podcast
Output directory: qwen_qlora_podcast_merged

1. Loading base model in 4-bit...

2. Loading PEFT model (adapter)...

3. Merging adapter into the base model...


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


✅ Merge complete.

4. Saving merged model to qwen_qlora_podcast_merged...

🎉 Merged model saved successfully to qwen_qlora_podcast_merged
You can now load this model directly for fast inference.


In [ ]:
%run 06_run_predictions.py

Running predictions in RESUME mode.
Evaluating Merged Fine-tuned Qwen Model
Found 2000 examples to evaluate
Creating new output column: finetuned Qwen 0.6

🚀 Loading merged model from qwen_qlora_podcast_merged...
Model and adapter loaded successfully
Starting inference...


Inferencing:   0%|          | 0/2000 [00:00<?, ?example/s]

Streaming output truncated to the last 5000 lines.
    __
    __
    __
    __
    __
✅ Processed 369/2000 examples
This is the response
You asked about David Lynch’s “The Elephant Man” and its screening at Brattle Theatre. Yes, the movie was released in 1980 and directed by Peter Weiss. It’s a classic Lynchian film, known for its sincerity and honest portrayal of compassion and humanity. The movie was originally planned as a 1980 film but was later truncated and renamed “The Elephant Man.” It’s a powerful exploration of cruelty and compassion, with scenes that highlight the struggles of real-life Joseph Merrick.
        The film is a timeless tale of compassion and humanity, with a deep exploration of the lives of those it portrays. The movie’s masterful visual language creates a dreamlike yet disturbing atmosphere, with scenes that beautifully convey the weight of existence for those who suffer.
        The film was directed by Peter Weiss, who co-wrote the screenplay with John Osbor